In [ ]:
!pip -q install --no-cache-dir datasets transformers evaluate accelerate scikit-learn matplotlib


In [ ]:
import numpy as np
import pandas as pd

from datasets import load_dataset, Dataset, DatasetDict
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer, DataCollatorWithPadding, set_seed)
import evaluate

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import matplotlib.pyplot as plt

set_seed(42)


In [ ]:
from huggingface_hub import HfApi

def load_turkish_movie_dataset():
    candidates = [
        "mkeskin/turkish_movie_sentiment",
    ]
    for ds_id in candidates:
        try:
            raw = load_dataset(ds_id)
            print("Loaded:", ds_id)
            return ds_id, raw
        except Exception as e:
            print(f"Failed: {ds_id} -> {type(e).__name__}: {e}")

    api = HfApi()
    results = list(api.list_datasets(search="turkish movie sentiment"))
    print("Search results:", len(results))

    for r in results[:40]:
        ds_id = r.id
        try:
            raw = load_dataset(ds_id)
            if "train" in raw:
                print("Auto-found:", ds_id)
                return ds_id, raw
        except Exception:
            pass

    raise RuntimeError("Uygun Türkçe film yorum dataset'i bulunamadı. (Gerekirse CSV yükleyip devam ederiz.)")

ds_id, raw = load_turkish_movie_dataset()
raw


In [ ]:
train_split = raw["train"]
cols = train_split.column_names
print("Columns:", cols)

def infer_columns(ds):
    cols = ds.column_names
    low = {c.lower(): c for c in cols}

    text_candidates = ["text", "review", "comment", "sentence", "content"]
    label_candidates = ["label", "rating", "stars", "score", "point", "puan", "yildiz"]

    text_col = None
    for k in text_candidates:
        if k in low:
            text_col = low[k]; break

    label_col = None
    for k in label_candidates:
        if k in low:
            label_col = low[k]; break

    # ilk string kolonu text yapıyoruz
    if text_col is None:
        for c in cols:
            try:
                if ds.features[c].dtype == "string":
                    text_col = c
                    break
            except Exception:
                pass

    if text_col is None or label_col is None:
        raise ValueError(f"Kolonlar tespit edilemedi. Bulunan text={text_col}, label={label_col}. Kolonlar={cols}")

    return text_col, label_col

text_col, label_col = infer_columns(train_split)
print("Detected text_col:", text_col, "| label_col:", label_col)

print("Sample text:", train_split[0][text_col])
print("Sample label:", train_split[0][label_col])


In [ ]:
import re
import numpy as np

def to_5class_label(x):
    #  int ise bunu
    if isinstance(x, (int, np.integer)):
        if 0 <= x <= 4:
            return int(x)
        if 1 <= x <= 5:
            return int(x - 1)

    #  float ise bunu
    if isinstance(x, (float, np.floating)):
        y = int(round(float(x)))
        y = max(1, min(5, y))
        return y - 1

    # string ise bunu
    if isinstance(x, str):
        s = x.strip()

        s = s.replace(" ", "")

        s = s.replace(",", ".")

        s = re.sub(r"[^0-9\.\-]", "", s)

        if s == "" or s == "." or s == "-":
            raise ValueError(f"Boş/Geçersiz label string: {x!r}")

        xf = float(s)
        y = int(round(xf))
        y = max(1, min(5, y))
        return y - 1


    raise ValueError(f"Label dönüştürülemedi: {x} ({type(x)})")


In [ ]:
ds_clean = standardize(train_split, text_col, label_col)
ds_clean[0]

In [ ]:
for i in range(10):
    print(train_split[i][label_col])

In [ ]:
tmp = ds_clean.to_pandas()
print(tmp["label"].value_counts().sort_index())

In [ ]:
df = ds_clean.to_pandas()

train_df, temp_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df["label"]
)
val_df, test_df = train_test_split(
    temp_df, test_size=0.5, random_state=42, stratify=temp_df["label"]
)

dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df.reset_index(drop=True)),
    "validation": Dataset.from_pandas(val_df.reset_index(drop=True)),
    "test": Dataset.from_pandas(test_df.reset_index(drop=True)),
})
dataset



In [ ]:
model_name = "dbmdz/bert-base-turkish-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, max_length=256)

tokenized = dataset.map(tokenize, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)


In [ ]:
acc = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": acc.compute(predictions=preds, references=labels)["accuracy"],
        "macro_f1": f1.compute(predictions=preds, references=labels, average="macro")["f1"],
    }


In [ ]:
training_args = TrainingArguments(
    output_dir="berturk_movie_sentiment",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    greater_is_better=True,
    fp16=True,
    logging_steps=50,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
test_metrics = trainer.evaluate(tokenized["test"])
print("TEST METRICS:", test_metrics)

pred = trainer.predict(tokenized["test"])
y_true = pred.label_ids
y_pred = np.argmax(pred.predictions, axis=-1)

print(classification_report(y_true, y_pred, digits=4))

In [ ]:
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(6,6))
plt.imshow(cm)
plt.title("Confusion Matrix (0-4 => stars 1-5)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.colorbar()
plt.show()


In [ ]:
test_texts = tokenized["test"]
wrong_idx = np.where(y_true != y_pred)[0]

print("Wrong predictions:", len(wrong_idx), "out of", len(y_true))

for i in wrong_idx[:10]:
    print("="*80)
    print("TRUE:", int(y_true[i]+1), "| PRED:", int(y_pred[i]+1))
    print(test_texts[int(i)]['text'][:500])